In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import sys
sys.path.append('./waveglow/')

import librosa
import numpy as np
import os
import glob
import json

import torch
from text import text_to_sequence, cmudict
from text.symbols import symbols
import commons
import attentions
import modules
import models
import utils

# load waveglow
waveglow_path = './waveglow/waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow = waveglow.remove_weightnorm(waveglow)
_ = waveglow.cuda().eval()
from apex import amp
waveglow, _ = amp.initialize(waveglow, [], opt_level="O3") # Try if you want to boost up synthesis speed.

/home/samuel/miniconda3/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/samuel/miniconda3/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/samuel/miniconda3/lib/python3.7/site-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the o

Selected optimization level O3:  Pure FP16 training.
Defaults for this optimization level are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O3
cast_model_type        : torch.float16
patch_torch_functions  : False
keep_batchnorm_fp32    : False
master_weights         : False
loss_scale             : 1.0


In [2]:
keep_punct = False

config_file = './configs/base.json'
    
model_dir = "./chp/"
hps = utils.get_hparams_from_file(config_file)

model = models.FlowGenerator(
    len(symbols),
    out_channels=hps.data.n_mel_channels,
    **hps.model).to("cuda")

#checkpoint_path = utils.latest_checkpoint_path(model_dir)
utils.load_checkpoint("./ex0/G_300.pth", model)
model.decoder.store_inverse() # do not calcuate jacobians for fast decoding
_ = model.eval()

cmu_dict = None

INFO:root:Loaded checkpoint './ex0/G_300.pth' (iteration 300)


In [3]:
tst_stn = " But Mrs. Solomons could not resist the temptation to dabble in stolen goods, and she was found shipping watches of the wrong category to New York. " # Adding spaces at the beginning and the end of utterance improves quality
sequence = np.array(text_to_sequence(tst_stn, ['english_cleaners'], cmu_dict, keep_punct))[None, :]
print("|".join([symbols[c] for c in sequence[0]]))
x_tst = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
x_tst_lengths = torch.tensor([x_tst.shape[1]]).cuda()

 |b|u|t| |m|i|s|e|s|s| |s|o|l|o|m|o|n|s| |c|o|u|l|d| |n|o|t| |r|e|s|i|s|t| |t|h|e| |t|e|m|p|t|a|t|i|o|n| |t|o| |d|a|b|b|l|e| |i|n| |s|t|o|l|e|n| |g|o|o|d|s| | |a|n|d| |s|h|e| |w|a|s| |f|o|u|n|d| |s|h|i|p|p|i|n|g| |w|a|t|c|h|e|s| |o|f| |t|h|e| |w|r|o|n|g| |c|a|t|e|g|o|r|y| |t|o| |n|e|w| |y|o|r|k| | 


In [5]:
with torch.no_grad():
    noise_scale = 0.3
    length_scale = 1.0
    y_pred, y_lengths = model(x_tst, x_tst_lengths, gen=True)
    audio = waveglow.infer(y_pred.half(), sigma=.666)
ipd.Audio(audio[0].clamp(-1,1).data.cpu().float().numpy(), rate=hps.data.sampling_rate)

torch.Size([1, 301]) torch.Size([1]) torch.Size([])
tensor([658], device='cuda:0') tensor(658, device='cuda:0')
